<a href="https://colab.research.google.com/github/KJM94/Team_project/blob/main/code/titanic_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2021/train.csv
/kaggle/input/tabular-playground-series-apr-2021/test.csv


In [ ]:
!pip install scikit-learn==0.23.2

     |████████████████████████████████| 6.8 MB 4.7 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.1 which is incompatible.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


https://www.kaggle.com/udbhavpangotra/tps-apr21-eda-model


https://www.kaggle.com/hiro5299834/tps-apr-2021-voting-pseudo-labeling

# KAGGLE 스터디 

In [ ]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
TARGET = 'Survived'

N_ESTIMATORS = 1000
N_SPLITS = 10
SEED = 2021
EARLY_STOPPING_ROUNDS = 100
VERBOSE = 100

In [ ]:
#랜덤 시드 생성
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

## 데이터 전처리

### lode data

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
#test_df['Survived'] = pd.read_csv("../input/submission-merged3/submission_merged3.csv")['Survived']

all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
#reset_index : 인덱스를 세팅한다. drop=True를 하면 인덱스를 세팅한걸 삭제함. 


In [ ]:
print('Rows and Columns in train dataset:', train_df.shape)
print('Rows and Columns in test dataset:', test_df.shape)

Rows and Columns in train dataset: (100000, 12)
Rows and Columns in test dataset: (100000, 11)


### 결측치 갯수 출력

In [ ]:
print('Missing values per columns in train dataset')
for col in train_df.columns:
    temp_col = train_df[col].isnull().sum()
    print(f'{col}: {temp_col}')
print()
print('Missing values per columns in test dataset')
for col in test_df.columns:
    temp_col = test_df[col].isnull().sum()
    print(f'{col}: {temp_col}')

Missing values per columns in train dataset
PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 3292
SibSp: 0
Parch: 0
Ticket: 4623
Fare: 134
Cabin: 67866
Embarked: 250

Missing values per columns in test dataset
PassengerId: 0
Pclass: 0
Name: 0
Sex: 0
Age: 3487
SibSp: 0
Parch: 0
Ticket: 5181
Fare: 133
Cabin: 70831
Embarked: 277


### Filling missing values

In [ ]:
#나이는 나이의 평균치로 채운다.
all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())

#cabin은 문자열을 분할하고, 제일 첫번째 글자를 따와서 넣는다. 결측치엔 X를 넣는다.
#strip() : 양쪽 공백을 지운다. 여기서느 x[0]외엔 다 지우는듯. 
all_df['Cabin'] = all_df['Cabin'].fillna('X').map(lambda x: x[0].strip())


#print(all_df['Ticket'].head(10))
#Ticket, fillna with 'X', split string and take first split 
#split() : 문자열 나누기. 디폴트는 ' '이고, 문자를 가진 데이터들이 전부 띄워쓰기로 구분되어있기때문에 가능. 
all_df['Ticket'] = all_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

#pclass에 따른 Fare의 평균을 구해서 dictionary형태로 만든다. 
fare_map = all_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
#fare의 결측치에 본인 행의 pclass 값을 넣고, 그 값을 fare 평균에 맵핑시킨다.  
all_df['Fare'] = all_df['Fare'].fillna(all_df['Pclass'].map(fare_map['Fare']))
#유독 높은 가격이나 낮은 가격이 있기때문에, 이상치의 영향을 줄이기 위해서 Fare에 log를 취해준다.
all_df['Fare'] = np.log1p(all_df['Fare'])


#항구의 결측치를 X로 채운다. 
all_df['Embarked'] = all_df['Embarked'].fillna('X')

#이름은 성만 사용한다.
all_df['Name'] = all_df['Name'].map(lambda x: x.split(',')[0])


In [ ]:
data_1=all_df.loc[all_df['Pclass']==1].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_1)
print()
data_2=all_df.loc[all_df['Pclass']==2].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_2)
print()
data_3=all_df.loc[all_df['Pclass']==3].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_3)
print()

Ticket
X             36336
PC            16814
C.A.            338
SC/Paris        334
SC/PARIS        260
W./C.           206
S.O.C.          192
S.C./PARIS      191
PP              186
F.C.            183
SC/AH           178
F.C.C.          167
STON/O          163
CA.             161
SOTON/O.Q.      123
A/4             115
A/5.            108
W.E.P.           94
WE/P             92
SOTON/OQ         87
CA               81
STON/O2.         81
A/5              70
C                67
A/4.             66
P/PP             66
SC               59
SOTON/O2         48
A./5.            46
S.O./P.P.        40
A.5.             33
AQ/4             27
A/S              23
SCO/W            19
S.P.             17
SC/A4            16
SW/PP            16
SC/A.3           15
S.O.P.           15
C.A./SOTON       14
A.               14
SO/C             14
S.C./A.4.        14
STON/OQ.         13
W/C              13
LP               11
S.W./PP          11
AQ/3.             8
Fa                7
A4.          

## 인코딩 

변수별로 인코딩을 다르게 해준다. 

In [ ]:
label_cols = ['Name', 'Ticket', 'Sex','Pclass','Embarked']
onehot_cols = [ 'Cabin',]
numerical_cols = [ 'Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
#라벨 인코딩 함수. c라는 매개변수를 받아서 맞게 트렌스폼 해준다. 
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

In [ ]:

#StandardScaler(): 평균을 제거하고 데이터를 단위 분산으로 조정한다. 
#그러나 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 되는 함수
scaler = StandardScaler()

onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
label_encoded_df = all_df[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
target_df = all_df[TARGET]

all_df = pd.concat([numerical_df, label_encoded_df,onehot_encoded_df, target_df], axis=1)
#all_df = pd.concat([numerical_df, label_encoded_df, target_df], axis=1)

## 모델링

In [ ]:
drop_list=['Survived', 'Parch']

In [ ]:
!pip install pycaret==2.2.3

     |████████████████████████████████| 249 kB 419 kB/s 
     |████████████████████████████████| 102 kB 3.5 MB/s 
     |████████████████████████████████| 14.2 MB 766 kB/s 
     |████████████████████████████████| 167 kB 9.4 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 79 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 9.1 MB/s 
     |████████████████████████████████| 1.7 MB 9.1 MB/s 
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ | done
    Preparing wheel metadata ... - done
     |████████████████████████████████| 1.7 MB 10.6 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=70d93f0d54782d162012e2d05104fb40e7f29daa8c090da2d57075c3b63f9784
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel fo

In [ ]:
from pycaret.utils import version
import sklearn
print("pycaret version:", version())
print("sklearn version:", sklearn.__version__)

pycaret version: 2.2.3
sklearn version: 0.23.2


In [ ]:
train = all_df.iloc[:100000, :]#0개~100000개
test = all_df.iloc[100000:, :] #100000개~ 
#iloc은 정수형 인덱싱
test = test.drop('Survived', axis=1) #test에서 종속변수를 드랍한다. 
model_results = pd.DataFrame()
folds = 5

In [ ]:
caret_train=train.drop('Parch', axis = 1)

In [ ]:

category_caret={'Sex':['0','1'],'Pclass':['0','1','2'], 'Embarked':['0','1','2','3']}
from pycaret.classification import *


setup(data = caret_train, 
      target = 'Survived',
      ordinal_features= category_caret,
      #numeric_imputation = 'Age','SibSp','Name','Ticket','Fare',
      fold=5,
      silent = True,
      session_id=1,
      #data_split_shuffle=True
      fold_shuffle=True

     )



,Description,Value
0,session_id,1
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(100000, 18)"
5,Missing Values,False
6,Numeric Features,14
7,Categorical Features,3
8,Ordinal Features,True
9,High Cardinality Features,False


([],
                 Age     SibSp      Fare   Name  Ticket  Sex  Pclass  Embarked  \
 0     -4.307127e-16  1.901268  0.134351  17441      49    1       0         2   
 1     -4.307127e-16 -0.539572 -0.533837   3063      49    1       2         2   
 2     -2.069149e+00  0.680848  1.070483  17798      14    1       2         2   
 3     -9.374220e-01 -0.539572 -0.555506  12742       0    1       2         2   
 4     -5.737175e-01 -0.539572 -1.023540   2335      49    1       2         2   
 ...             ...       ...       ...    ...     ...  ...     ...       ...   
 99995  1.669127e+00 -0.539572 -0.434567   1590      21    0       1         0   
 99996  1.911597e+00 -0.539572 -0.698959   2992      49    1       1         2   
 99997  1.536915e-01 -0.539572 -0.802137   4219      49    1       2         2   
 99998  1.002335e+00 -0.539572  0.259408   3941      49    1       2         2   
 99999  1.244805e+00 -0.539572 -0.492531   7055      49    1       2         2   
 
        C

In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,pycaret.internal.tunable.TunableMLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
#lgbms = [create_model('lightgbm', learning_rate = i) for i in np.arange(0.1, 1, 0.1)]

In [ ]:
#best_model = compare_models(sort = 'Accuracy', n_select = 3)

In [ ]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7825,0.8494,0.7359,0.7513,0.7435,0.5547,0.5548
1,0.7801,0.8490,0.7437,0.7431,0.7434,0.5510,0.5510
2,0.7863,0.8547,0.7467,0.7525,0.7496,0.5632,0.5632
3,0.7796,0.8490,0.7391,0.7447,0.7419,0.5496,0.5497
4,0.7833,0.8502,0.7475,0.7469,0.7472,0.5577,0.5577
Mean,0.7824,0.8505,0.7426,0.7477,0.7451,0.5552,0.5553
SD,0.0024,0.0022,0.0045,0.0037,0.0029,0.0049,0.0049


In [ ]:
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7786,0.8480,0.7287,0.7479,0.7382,0.5464,0.5465
1,0.7774,0.8474,0.7394,0.7405,0.7399,0.5453,0.5453
2,0.7826,0.8532,0.7417,0.7486,0.7452,0.5557,0.5557
3,0.7761,0.8477,0.7354,0.7403,0.7379,0.5425,0.5425
4,0.7835,0.8499,0.7410,0.7504,0.7457,0.5572,0.5572
Mean,0.7796,0.8492,0.7373,0.7456,0.7414,0.5494,0.5495
SD,0.0029,0.0022,0.0048,0.0043,0.0034,0.0059,0.0059


In [ ]:
# ada = create_model('ada')

In [ ]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7786,0.8482,0.7417,0.7416,0.7416,0.5480,0.5480
1,0.7800,0.8478,0.7512,0.7394,0.7452,0.5517,0.5517
2,0.7851,0.8535,0.7554,0.7461,0.7507,0.5618,0.5619
3,0.7754,0.8480,0.7426,0.7356,0.7391,0.5419,0.5419
4,0.7826,0.8501,0.7484,0.7451,0.7468,0.5562,0.5562
Mean,0.7803,0.8495,0.7479,0.7416,0.7447,0.5519,0.5519
SD,0.0033,0.0021,0.0052,0.0038,0.0040,0.0068,0.0068


In [ ]:
ada = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7726,0.8423,0.6943,0.7552,0.7235,0.5310,0.5324
1,0.7749,0.8419,0.7114,0.7501,0.7302,0.5373,0.5378
2,0.7793,0.8469,0.7129,0.7576,0.7346,0.5460,0.5467
3,0.7697,0.8408,0.6957,0.7489,0.7213,0.5256,0.5266
4,0.7789,0.8442,0.7092,0.7589,0.7332,0.5448,0.5457
Mean,0.7751,0.8432,0.7047,0.7542,0.7286,0.5369,0.5379
SD,0.0037,0.0021,0.0080,0.0040,0.0053,0.0079,0.0077


In [ ]:
tuned_lightgbm = tune_model(lightgbm, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7793,0.8496,0.7382,0.7444,0.7413,0.5488,0.5489
1,0.7797,0.8492,0.7439,0.7424,0.7431,0.5503,0.5503
2,0.7858,0.8547,0.7489,0.7505,0.7497,0.5625,0.5625
3,0.7777,0.8493,0.7467,0.7376,0.7422,0.5468,0.5469
4,0.7825,0.8508,0.7429,0.7477,0.7453,0.5555,0.5555
Mean,0.7810,0.8507,0.7441,0.7445,0.7443,0.5528,0.5528
SD,0.0028,0.0021,0.0036,0.0044,0.0030,0.0056,0.0056


In [ ]:
# tuned_ada = tune_model(ada)

In [ ]:
tuned_catboost = tune_model(catboost, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7815,0.8501,0.7455,0.7447,0.7451,0.5539,0.5539
1,0.7802,0.8490,0.7485,0.7410,0.7448,0.5518,0.5518
2,0.7861,0.8554,0.7539,0.7486,0.7512,0.5636,0.5636
3,0.7786,0.8494,0.7462,0.7395,0.7428,0.5485,0.5486
4,0.7830,0.8509,0.7482,0.7460,0.7471,0.5570,0.5570
Mean,0.7819,0.8510,0.7485,0.7439,0.7462,0.5550,0.5550
SD,0.0025,0.0023,0.0029,0.0033,0.0028,0.0051,0.0051


In [ ]:
tuned_gbc = tune_model(gbc, optimize= 'Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7801,0.8485,0.7365,0.7466,0.7415,0.5502,0.5502
1,0.7799,0.8485,0.7414,0.7440,0.7427,0.5504,0.5504
2,0.7839,0.8545,0.7442,0.7496,0.7469,0.5584,0.5584
3,0.7786,0.8494,0.7359,0.7445,0.7402,0.5474,0.5474
4,0.7837,0.8511,0.7414,0.7506,0.7460,0.5577,0.5577
Mean,0.7813,0.8504,0.7399,0.7471,0.7435,0.5528,0.5528
SD,0.0021,0.0023,0.0032,0.0027,0.0026,0.0044,0.0044


In [ ]:
tuned_ada = tune_model(ada, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7758,0.8421,0.7015,0.7572,0.7283,0.5379,0.5391
1,0.7759,0.8415,0.7087,0.7534,0.7304,0.5389,0.5396
2,0.7802,0.8464,0.7262,0.7522,0.7390,0.5493,0.5495
3,0.7708,0.8404,0.7016,0.7478,0.7240,0.5283,0.5291
4,0.7790,0.8431,0.7125,0.7572,0.7342,0.5453,0.5461
Mean,0.7763,0.8427,0.7101,0.7536,0.7312,0.5400,0.5407
SD,0.0033,0.0020,0.0091,0.0035,0.0051,0.0072,0.0070


In [ ]:
test = all_df.iloc[100000:, :]
X_test = test.drop(drop_list, axis=1)
X_test.head()

,Age,SibSp,Fare,Name,Ticket,Sex,Pclass,Embarked,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X
100000,-0.937422,-0.539572,0.949786,10830,49,1,2,2,0,0,0,0,0,0,0,0,1
100001,1.123570,-0.539572,-1.273379,17134,49,0,2,2,0,0,0,0,0,0,0,0,1
100002,-0.937422,-0.539572,0.481059,9978,49,0,0,0,0,1,0,0,0,0,0,0,0
100003,-0.573717,-0.539572,-0.563310,13303,49,1,1,2,0,0,0,0,0,0,0,0,1
100004,-1.058657,-0.539572,0.125497,4406,49,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
#best_specific = compare_models(include = ['lightgbm', 'ada'])

In [ ]:
#best_specific = compare_models(include = ['lightgbm', 'gbc'])

In [ ]:
#best_specific = compare_models(include = ['lightgbm', 'ada', 'gbc'])

Soft voting

In [ ]:
def create_submission(model, test, test_passenger_id, model_name):
    y_pred_test = model.predict_proba(test)[:, 1]
    submission = pd.DataFrame({'PassengerId' : test_passenger_id,
                              'Survived' : (y_pred_test >= 0.5).astype(int),
                              })
    submission.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [ ]:
test_pred_catboost = create_submission(
    tuned_catboost, X_test, test_df["PassengerId"], "catboost"
)

test_pred_gbc = create_submission(
    tuned_gbc, X_test, test_df["PassengerId"], "gbc"
 )

test_pred_lightgbm = create_submission(
    tuned_lightgbm, X_test, test_df["PassengerId"], "gbc"
 )

test_pred_ada = create_submission(
    tuned_ada, X_test, test_df["PassengerId"], "gbc"
 )

In [ ]:
test_pred_merged = (
    test_pred_catboost +
    test_pred_gbc +
    test_pred_ada +
    test_pred_lightgbm
)

test_pred_merged = np.round(test_pred_merged / 4)

In [ ]:
submission = pd.DataFrame(
    {
        'PassengerId' : test_df["PassengerId"],
        'Survived' : test_pred_merged.astype(int),
    }
)

submission.to_csv(f"submission_merged.csv", index=False)

## not pseudo

In [ ]:
'''train = all_df.iloc[:100000, :]#0개~100000개
test = all_df.iloc[100000:, :] #100000개~ 
#iloc은 정수형 인덱싱
test = test.drop('Survived', axis=1) #test에서 종속변수를 드랍한다. 
model_results = pd.DataFrame()
folds = 5'''

"train = all_df.iloc[:100000, :]#0개~100000개\ntest = all_df.iloc[100000:, :] #100000개~ \n#iloc은 정수형 인덱싱\ntest = test.drop('Survived', axis=1) #test에서 종속변수를 드랍한다. \nmodel_results = pd.DataFrame()\nfolds = 5"

In [ ]:
'''y= train.loc[:,'Survived']
X= train.drop(drop_list,axis=1)'''

"y= train.loc[:,'Survived']\nX= train.drop(drop_list,axis=1)"

## pseudo

In [ ]:
# y=all_df.loc[:,'Survived']
# X=all_df.drop('Survived',axis=1)

In [ ]:
'''X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.25, random_state=21)
'''

'X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.25, random_state=21)\n'

In [ ]:
'''from sklearn import metrics  
from sklearn.metrics import accuracy_score
import numpy as np'''

'from sklearn import metrics  \nfrom sklearn.metrics import accuracy_score\nimport numpy as np'

In [ ]:
'''params = {
    'metric': 'binary_logloss',
    'n_estimators': N_ESTIMATORS,
    'objective': 'binary',
    'random_state': SEED,
    'learning_rate': 0.01,
    'min_child_samples': 150,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 20,
    'max_depth': 16,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 2,
    'max_bin': 240,
}'''

"params = {\n    'metric': 'binary_logloss',\n    'n_estimators': N_ESTIMATORS,\n    'objective': 'binary',\n    'random_state': SEED,\n    'learning_rate': 0.01,\n    'min_child_samples': 150,\n    'reg_alpha': 3e-5,\n    'reg_lambda': 9e-2,\n    'num_leaves': 20,\n    'max_depth': 16,\n    'colsample_bytree': 0.8,\n    'subsample': 0.8,\n    'subsample_freq': 2,\n    'max_bin': 240,\n}"

In [ ]:
'''lgbm_model=lgb.LGBMClassifier(**params)
lgbm_model.fit(X_train,y_train)
lgbm_pred=lgbm_model.predict(X_valid)

lgbm_R2=metrics.accuracy_score(y_valid,lgbm_pred)
#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))
print('R2 : ',lgbm_R2)
#print("RMSE : ", lgbm_rmse)'''

'lgbm_model=lgb.LGBMClassifier(**params)\nlgbm_model.fit(X_train,y_train)\nlgbm_pred=lgbm_model.predict(X_valid)\n\nlgbm_R2=metrics.accuracy_score(y_valid,lgbm_pred)\n#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))\nprint(\'R2 : \',lgbm_R2)\n#print("RMSE : ", lgbm_rmse)'

In [ ]:
'''print(len(X_train.columns))
print(X_train.columns)'''

'print(len(X_train.columns))\nprint(X_train.columns)'

In [ ]:
'''def cal_adjust_r2(r2):
    n=80000
    k= len(X_train.columns)
    temp=(1-r2)*(n-1)
    temp2=n-k-1
    ad_r2=1-(temp/temp2)
    return ad_r2'''

'def cal_adjust_r2(r2):\n    n=80000\n    k= len(X_train.columns)\n    temp=(1-r2)*(n-1)\n    temp2=n-k-1\n    ad_r2=1-(temp/temp2)\n    return ad_r2'

In [ ]:
'''ad_r2_lgbm=cal_adjust_r2(lgbm_R2)
print(ad_r2_lgbm)'''

'ad_r2_lgbm=cal_adjust_r2(lgbm_R2)\nprint(ad_r2_lgbm)'

In [ ]:
'''#NOT Pseudo
train_kf_feature=train.drop(drop_list,axis=1)
train_kf_label=train.loc[:,'Survived']'''

"#NOT Pseudo\ntrain_kf_feature=train.drop(drop_list,axis=1)\ntrain_kf_label=train.loc[:,'Survived']"

In [ ]:
#Pseudo
# train_kf_feature=all_df.drop(drop_list,axis=1)
# train_kf_label=all_df.loc[:,'Survived']

In [ ]:
'''n_iter=0
kfold=StratifiedKFold(n_splits=5)
cv_accuracy=[]
feature_importances = pd.DataFrame()

for train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):

    X_train=train_kf_feature.iloc[train_idx]
    X_test=train_kf_feature.iloc[test_idx]
    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]
    #학습 진행
    lgbm_model.fit(X_train,y_train)
    #예측
    fold_pred=lgbm_model.predict(X_test)
    
    #정확도
    n_iter+=1
    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)
    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".
          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))
    cv_accuracy.append(fold_accuracy)
    
    #중요도 
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = lgbm_model.feature_name_
    fi_tmp["importance"] = lgbm_model.feature_importances_
    feature_importances = feature_importances.append(fi_tmp)

print('\n 평균 검증 정확도 : ',np.mean(cv_accuracy))
    '''

'n_iter=0\nkfold=StratifiedKFold(n_splits=5)\ncv_accuracy=[]\nfeature_importances = pd.DataFrame()\n\nfor train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):\n\n    X_train=train_kf_feature.iloc[train_idx]\n    X_test=train_kf_feature.iloc[test_idx]\n    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]\n    #학습 진행\n    lgbm_model.fit(X_train,y_train)\n    #예측\n    fold_pred=lgbm_model.predict(X_test)\n    \n    #정확도\n    n_iter+=1\n    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)\n    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".\n          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))\n    cv_accuracy.append(fold_accuracy)\n    \n    #중요도 \n    fi_tmp = pd.DataFrame()\n    fi_tmp["feature"] = lgbm_model.feature_name_\n    fi_tmp["importance"] = lgbm_model.feature_importances_\n    feature_importances = feature_importances.append(fi_tmp)\n\nprint(\'\n 평균 검증 정확도 : \',np.mean(cv_accuracy))\n    '

In [ ]:
'''order = list(feature_importances.groupby("feature").
             mean().sort_values("importance", ascending=False).index)
plt.figure(figsize=(10, 10))
sns.barplot(x="importance", y="feature", data=feature_importances, order=order)
plt.title("{} importance".format("LGBMRegressor"))
plt.tight_layout()'''

'order = list(feature_importances.groupby("feature").\n             mean().sort_values("importance", ascending=False).index)\nplt.figure(figsize=(10, 10))\nsns.barplot(x="importance", y="feature", data=feature_importances, order=order)\nplt.title("{} importance".format("LGBMRegressor"))\nplt.tight_layout()'

### CATBoost


In [ ]:
'''params_cat = {
    'bootstrap_type': 'Poisson',
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'random_seed': SEED,
    'task_type': 'GPU',
    'max_depth': 8,
    'learning_rate': 0.01,
    'n_estimators': N_ESTIMATORS,
    'max_bin': 280,
    'min_data_in_leaf': 64,
    'l2_leaf_reg': 0.01,
    'subsample': 0.8
}'''

"params_cat = {\n    'bootstrap_type': 'Poisson',\n    'loss_function': 'Logloss',\n    'eval_metric': 'Logloss',\n    'random_seed': SEED,\n    'task_type': 'GPU',\n    'max_depth': 8,\n    'learning_rate': 0.01,\n    'n_estimators': N_ESTIMATORS,\n    'max_bin': 280,\n    'min_data_in_leaf': 64,\n    'l2_leaf_reg': 0.01,\n    'subsample': 0.8\n}"

In [ ]:
'''#새로운 트레인 valid 셋
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.25, random_state=21)
''''''

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-56-5f35148964f0>, line 3)

In [ ]:
'''
cat_model=ctb.CatBoostClassifier(**params_cat)
cat_model.fit(X_train, y_train,verbose=300)
cat_pred=cat_model.predict(X_valid)
print("\n정확도: ", metrics.accuracy_score(y_valid, cat_pred))
cat_R2=metrics.accuracy_score(y_valid,cat_pred)
#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))
print('R2 : ',cat_R2)'''

'\ncat_model=ctb.CatBoostClassifier(**params_cat)\ncat_model.fit(X_train, y_train,verbose=300)\ncat_pred=cat_model.predict(X_valid)\nprint("\n정확도: ", metrics.accuracy_score(y_valid, cat_pred))\ncat_R2=metrics.accuracy_score(y_valid,cat_pred)\n#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))\nprint(\'R2 : \',cat_R2)'

In [ ]:
'''cv_accuracy=[]
feature_importances = pd.DataFrame()

for train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):

    X_train=train_kf_feature.iloc[train_idx]
    X_test=train_kf_feature.iloc[test_idx]
    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]
    #학습 진행
    cat_model.fit(X_train,y_train,verbose=500)
    #예측
    fold_pred=cat_model.predict(X_test)
    
    #정확도
    n_iter+=1
    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)
    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".
          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))
    cv_accuracy.append(fold_accuracy)
    
    #중요도 . lgbm이랑 명령어가 다르다.
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = X_test.columns.to_list()
    fi_tmp["importance"] = cat_model.get_feature_importance()
    feature_importances = feature_importances.append(fi_tmp)

print('\n 평균 검증 정확도 : ',np.mean(cv_accuracy))'''

'cv_accuracy=[]\nfeature_importances = pd.DataFrame()\n\nfor train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):\n\n    X_train=train_kf_feature.iloc[train_idx]\n    X_test=train_kf_feature.iloc[test_idx]\n    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]\n    #학습 진행\n    cat_model.fit(X_train,y_train,verbose=500)\n    #예측\n    fold_pred=cat_model.predict(X_test)\n    \n    #정확도\n    n_iter+=1\n    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)\n    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".\n          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))\n    cv_accuracy.append(fold_accuracy)\n    \n    #중요도 . lgbm이랑 명령어가 다르다.\n    fi_tmp = pd.DataFrame()\n    fi_tmp["feature"] = X_test.columns.to_list()\n    fi_tmp["importance"] = cat_model.get_feature_importance()\n    feature_importances = feature_importances.append(fi_tmp)\n\nprint(\'\n 평균 검증 정확도 : \',np.mean(cv_accuracy))'

In [ ]:
'''# just to get ideas to improve
order = list(feature_importances.groupby("feature").mean().sort_values("importance", ascending=False).index)
plt.figure(figsize=(10, 10))
sns.barplot(x="importance", y="feature", data=feature_importances, order=order)
plt.title("{} importance".format("CatBoostClassifier"))
plt.tight_layout()'''

'# just to get ideas to improve\norder = list(feature_importances.groupby("feature").mean().sort_values("importance", ascending=False).index)\nplt.figure(figsize=(10, 10))\nsns.barplot(x="importance", y="feature", data=feature_importances, order=order)\nplt.title("{} importance".format("CatBoostClassifier"))\nplt.tight_layout()'

## Submission


In [ ]:
'''def create_submission(model, test, test_passenger_id, model_name):
    y_pred_test = model.predict_proba(test)[:, 1]
    submission = pd.DataFrame(
        {
            'PassengerId': test_passenger_id, 
            'Survived': (y_pred_test >= 0.5).astype(int),
        }
    )
    submission.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test'''

'def create_submission(model, test, test_passenger_id, model_name):\n    y_pred_test = model.predict_proba(test)[:, 1]\n    submission = pd.DataFrame(\n        {\n            \'PassengerId\': test_passenger_id, \n            \'Survived\': (y_pred_test >= 0.5).astype(int),\n        }\n    )\n    submission.to_csv(f"submission_{model_name}.csv", index=False)\n    \n    return y_pred_test'

In [ ]:
'''test_df.head()'''

'test_df.head()'

In [ ]:
'''#X_test=test.drop('Pclass',axis=1)
test = all_df.iloc[100000:, :] #100000개~ 
X_test=test.drop(drop_list,axis=1)
X_test.head()'''

"#X_test=test.drop('Pclass',axis=1)\ntest = all_df.iloc[100000:, :] #100000개~ \nX_test=test.drop(drop_list,axis=1)\nX_test.head()"

In [ ]:
'''test_pred_lightgbm = create_submission(
    lgbm_model, X_test, test_df["PassengerId"], "lightgbm"
)
test_pred_catboost = create_submission(
    cat_model, X_test, test_df["PassengerId"], "catboost"
)'''

'test_pred_lightgbm = create_submission(\n    lgbm_model, X_test, test_df["PassengerId"], "lightgbm"\n)\ntest_pred_catboost = create_submission(\n    cat_model, X_test, test_df["PassengerId"], "catboost"\n)'

In [ ]:
'''test_pred_merged = (

    test_pred_lightgbm + 
    test_pred_catboost 
)
test_pred_merged = np.round(test_pred_merged / 2)'''

'test_pred_merged = (\n\n    test_pred_lightgbm + \n    test_pred_catboost \n)\ntest_pred_merged = np.round(test_pred_merged / 2)'

In [ ]:
'''submission = pd.DataFrame(
    {
        'PassengerId': test_df["PassengerId"], 
        'Survived': test_pred_merged.astype(int),
    }
)
submission.to_csv(f"submission_merged3.csv", index=False)'''

'submission = pd.DataFrame(\n    {\n        \'PassengerId\': test_df["PassengerId"], \n        \'Survived\': test_pred_merged.astype(int),\n    }\n)\nsubmission.to_csv(f"submission_merged3.csv", index=False)'